# Seleccion de variables

El obejetivo es seleccionar las mejores variables para entrenar los modelos.

**Metodos y Paquete :**

- [Boruta](https://pypi.org/project/Boruta/)
- [Ejemplo de uso boruta](https://towardsdatascience.com/feature-selection-with-boruta-in-python-676e3877e596)
- [Mutual Information](https://towardsdatascience.com/select-features-for-machine-learning-model-with-mutual-information-534fe387d5c8)


In [2]:
import numpy as np
import warnings
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import KFold
from tqdm import tqdm
import json

In [3]:
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format #evita que muestre notacion cientifica
pd.set_option('display.max_columns', None)
np.set_printoptions(suppress=True) #evita mostrar notacion cientifica

In [4]:
def feature_selection_by_correlation(x_train, y_train, variables,method='pearson', th=0.9):
    print('Calculando Correlación Entre Variables')
    x_train['target'] = y_train
    df_corr = x_train[variables + ['target']].corr(method=method)
    # Buscar variables mas correlacionadas
    vars_to_drop_corr = []
    for x in variables:
        for y in variables:
            if x != y:
#                 print(x,y)
                c_value = df_corr[x][y]
                if np.abs(c_value) > th:
                    corr_x_t = np.abs(df_corr[x]['target'])
                    corr_y_t = np.abs(df_corr[y]['target'])
                    if corr_x_t > corr_y_t:
                        vars_to_drop_corr.append(y)
    x_train.drop(columns=['target'],inplace=True)
    print('Eliminando Variables Altamente Correlacionadas')
    variables = [x for x in variables if x not in vars_to_drop_corr]
    return variables

def feature_selection_by_constant(x_train, y_train, variables, th=0.99):
    num_rows = x_train.shape[0]
    allLabels = variables
    constant_per_feature = {label: x_train[label].value_counts().iloc[0]/num_rows for label in allLabels}
    variables_to_drop = [label for label in allLabels if constant_per_feature [label] > th]
    variables = [x for x in variables if x not in variables_to_drop]
    return variables

def feature_selection_by_boruta(X_train, y_train, N=10):
    d = {}
    X_train = X_train.copy()
    y_train = y_train.copy()
    for i in tqdm(range(N),total=N):
        # Hacemos eliminacion de variables por Boruta.
        X_train['random'] = [np.random.randn() for i in range(len(X_train))]
        rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=8)
        feat_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=i, perc=100)
        feat_selector.fit(X_train.values, y_train.values)
        ranking = pd.DataFrame({'col': X_train.columns, 'ranking': feat_selector.ranking_}).sort_values(
            ['ranking']).reset_index(drop=True)
        variables = ranking.head(ranking.loc[ranking.col == 'random'].index.values[0]).col.values
        d[i] = variables

    E = {}
    for i in d.keys():
        for j in d[i]:
            if j not in E.keys():
                E[j] = 1
            else:
                E[j] += 1

    variables = [k for k in E.keys() if E[k] >= N // 2]
    return variables

def feature_selection_by_mutual_information(X_train, y_train, th=0.001, top_n = None):
    columns = X_train.columns.tolist()
    mi = mutual_info_classif(X_train, y_train)
    if th==None:
        th = np.quantile(mi,q=0.25)
    
    variables_mi  = dict(zip(columns,mi))
    variables_mi = list(sorted(variables_mi.items(), key=lambda item: item[1]))
    if top_n == None:
        return [x[0] for x in variables_mi if x[1]>=th]
    else:
        return [x[0] for x in res[-top_n:]]
    
def feature_selection_by_boruta2(X, y, n_splits=5,prop = 0.5):
    min_cant = int(n_splits*prop)
    # Initiate variables
    feature_labels = list(X.columns)
    selected_features_mask = np.zeros(len(feature_labels))

    #Get K-folds indices
    kf = KFold(n_splits = n_splits)
    kf.get_n_splits(X)

    # K-fold cross validation
    for train_index, val_index in tqdm(kf.split(X), total=kf.get_n_splits(X)):
        print(len(train_index),len(val_index))
        # Get train fold data
        X_train_fold = X.iloc[train_index, :]
        y_train_fold = y.iloc[train_index]
        # Define Boruta feature selection method
        rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=8)
        feat_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=None, perc=100)
    #     feat_selector = BorutaPy(**params)
        # Find all relevant features
        feat_selector.fit(X_train_fold.values, y_train_fold)

        # Boruta selected feature mask
        selected_features_temp = feat_selector.support_

        # Update selected relevant features
        selected_features_mask = selected_features_mask+selected_features_temp

    # Boruta selected feature labels
    labels = [feature_labels[ii] for ii in range(len(feature_labels)) if  selected_features_mask[ii]>= min_cant]
    
    return labels       
    

## Leer Archivos

In [12]:
print("Read data - Inicio")

Read data - Inicio


In [5]:
df = pd.read_parquet('tmp/df_tsfel_variables.parquet')

In [6]:
df.head()

uc  12_anterior  11_anterior  10_anterior  9_anterior  8_anterior  \
0   100035       153.00       125.00       117.00      120.00      128.00   
1   100043       253.00       246.00       259.00      262.00      254.00   
2    10008       187.00         0.00       205.00      187.00      178.00   
3  1000912       657.00       576.00       636.00      595.00      659.00   
4  1000996        70.00        61.00        61.00       69.00       85.00   

   7_anterior  6_anterior  5_anterior  4_anterior  3_anterior  2_anterior  \
0       80.00      105.00      123.00      101.00      111.00       99.00   
1      184.00      188.00      189.00      201.00      204.00      189.00   
2      194.00      200.00      188.00      168.00        0.00        0.00   
3      598.00      616.00      657.00      613.00      613.00      633.00   
4       97.00       96.00       95.00      100.00      104.00       90.00   

   1_anterior date_fizcalizacion  cant_ttarifa departamento  \
0       96.00         2022-01-01             1    guatemala   
1      188.00         2022-01-01             1    guatemala   
2       26.00         2022-01-01             1    guatemala   
3      616.00         2022-01-01             1    guatemala   
4       76.00         2022-01-01             1    guatemala   

            municipio     zona tipo_tarifa contrato       date  cod_mat  \
0         villa_nueva   zona_3          TS   100035 2022-01-01  41-0069   
1           guatemala   zona_4          TS   100043 2022-01-01  41-0081   
2           guatemala  zona_10          TS    10008 2022-01-01  41-0083   
3               mixco   zona_4         TNS  1000912 2022-01-01  41-0083   
4  san_jose_del_golfo   zona_0          TS  1000996 2022-01-01  41-0084   

   is_fraud  cant_null  eliminar unidad_de_lectura codigo_postal  \
0         0          0         0           708-008          1064   
1         0          0         0           605-015          1004   
2         0          0         0           603-001          1010   
3         0          0         0           612-039          1057   
4         0          0         0           813-035          1053   

  fecha_de_alta no_de_poste tarfia  multiplicador actividad_economica  \
0    2001-07-11      238366    R11           1.00              83_009   
1    2001-07-06      135771    C11           1.00              85_009   
2    2001-07-04      107349    C11           1.00              83_002   
3    2008-07-24      518933    C11           1.00              49_005   
4    2008-07-24      386109    R11           1.00              83_009   

   kw_cont medidor_interior          folio  nivel_de_tension  index  \
0     0.00         sin_dato  708-10329-000              2.00      0   
1     0.00         sin_dato  605-11884-000              2.00      1   
2     0.00                X  603-02018-000              2.00      2   
3     0.00         sin_dato  612-53632-000              2.00      3   
4     0.00         sin_dato  813-61192-000              2.00      4   

   0_ECDF Percentile Count_0  0_ECDF Percentile Count_1  0_ECDF Percentile_0  \
0                       2.00                       9.00                96.00   
1                       2.00                       9.00               188.00   
2                       2.00                       9.00                 0.00   
3                       2.00                       9.00               595.00   
4                       2.00                       9.00                61.00   

   0_ECDF Percentile_1  0_ECDF_0  0_ECDF_1  0_ECDF_2  0_ECDF_3  0_ECDF_4  \
0               123.00      0.08      0.17      0.25      0.33      0.42   
1               253.00      0.08      0.17      0.25      0.33      0.42   
2               188.00      0.08      0.17      0.25      0.33      0.42   
3               636.00      0.08      0.17      0.25      0.33      0.42   
4                96.00      0.08      0.17      0.25      0.33      0.42   

   0_ECDF_5  0_ECDF_6  0_ECDF_7  0_ECDF

In [7]:
df.shape

(30932, 207)

In [8]:
df.rename(columns={'id_usuario':'uc'}, inplace=True)
df.rename(columns={'id':'index'}, inplace=True)
df.rename(columns={'is_fraud':'target'}, inplace=True)

In [9]:
cols = ['uc','index','date_fizcalizacion','target']
cols_consumo = [f'{i}_anterior' for i in range(12, 0, -1)]
other_cols = ['cant_ttarifa', 'date', 'target', 'cant_null', 'eliminar', 'fecha_de_alta', 'multiplicador', 'kw_cont', 'nivel_de_tension','index']

In [10]:
cols_tsfel = [x for x in df.columns if x not in df.select_dtypes('O').columns.tolist() + cols + cols_consumo + other_cols]

In [11]:
df = df[cols + cols_tsfel]

In [13]:
df.head()

uc  index date_fizcalizacion  target  0_ECDF Percentile Count_0  \
0   100035      0         2022-01-01       0                       2.00   
1   100043      1         2022-01-01       0                       2.00   
2    10008      2         2022-01-01       0                       2.00   
3  1000912      3         2022-01-01       0                       2.00   
4  1000996      4         2022-01-01       0                       2.00   

   0_ECDF Percentile Count_1  0_ECDF Percentile_0  0_ECDF Percentile_1  \
0                       9.00                96.00               123.00   
1                       9.00               188.00               253.00   
2                       9.00                 0.00               188.00   
3                       9.00               595.00               636.00   
4                       9.00                61.00                96.00   

   0_ECDF_0  0_ECDF_1  0_ECDF_2  0_ECDF_3  0_ECDF_4  0_ECDF_5  0_ECDF_6  \
0      0.08      0.17      0.25      0.33      0.42      0.50      0.58   
1      0.08      0.17      0.25      0.33      0.42      0.50      0.58   
2      0.08      0.17      0.25      0.33      0.42      0.50      0.58   
3      0.08      0.17      0.25      0.33      0.42      0.50      0.58   
4      0.08      0.17      0.25      0.33      0.42      0.50      0.58   

   0_ECDF_7  0_ECDF_8  0_ECDF_9  0_Histogram_0  0_Histogram_1  0_Histogram_2  \
0      0.67      0.75      0.83           0.00           0.00           0.00   
1      0.67      0.75      0.83           0.00           0.00           0.00   
2      0.67      0.75      0.83           0.00           0.00           0.00   
3      0.67      0.75      0.83           0.00           0.00           0.00   
4      0.67      0.75      0.83           0.00           0.00           0.00   

   0_Histogram_3  0_Histogram_4  0_Histogram_5  0_Histogram_6  0_Histogram_7  \
0           0.00           0.00           0.00           0.00           0.00   
1           0.00           0.00           0.00           0.00           0.00   
2           0.00           0.00           3.00           0.00           0.00   
3           0.00           0.00           0.00           0.00           0.00   
4           0.00           0.00           0.00           0.00           0.00   

   0_Histogram_8  0_Histogram_9  0_Interquartile range  0_Kurtosis  0_Max  \
0           0.00           0.00                  23.00        0.09 153.00   
1           0.00           0.00                  64.50       -1.77 262.00   
2           0.00           0.00                 170.00       -1.46 205.00   
3           0.00           0.00                  32.00       -0.97 659.00   
4           0.00           0.00                  26.50       -1.44 104.00   

   0_Mean  0_Mean absolute deviation  0_Median  0_Median absolute deviation  \
0  113.17                      14.50    114.00                        12.00   
1  218.08                      30.60    202.50                        16.50   
2  127.75                      80.83    182.50                        16.00   
3  622.42                      21.65    616.00                        19.00   
4   83.67                      13.56     87.50                        12.00   

   0_Min  0_Root mean square  0_Skewness  0_Standard deviation  0_Variance  \
0  80.00              114.60        0.32                 18.07      326.64   
1 184.00              220.37        0.30                 31.68     1003.74   
2   0.00              154.25       -0.69                 86.45     7473.85   
3 576.00              622.94       -0.01                 25.48      649.08   
4  61.00               84.99       -0.27                 14.91      222.39   

   0_Absolute energy  0_Area under the curve  0_Autocorrelation  0_Centroid  \
0          157600.00                   12.33          157600.00        0.05   
1          582769.00                   23.96          582769.00        0.05   
2          285527.00                   14.27          2

In [14]:
# Eliminamos las variables tsfel con nulls
df = df.dropna(axis=1)

In [15]:
df.shape

(30932, 174)

In [16]:
print("Read data - Fin")

Read data - Fin


### Particionar dataset

**Hacer la seleccion de variables con el conjunto de train**

In [21]:
#Particionar por fecha
# df_train = df[df.date_fizcalizacion<='2021-03-01'].copy()
# y_train = df_train.target.copy()
# df_train = df_train.drop(columns=['target'])

# df_val = df[df.date_fizcalizacion.between('2021-04-01','2021-09-01')].copy()
# y_val = df_val.target.copy()
# df_val = df_val.drop(columns=['target'])

# df_test = df[df.date_fizcalizacion>'2021-09-01'].copy()
# y_test = df_test.target.copy()
# df_test = df_test.drop(columns=['target'])

# print(df_train.shape)
# print(df_val.shape)
# print(df_test.shape)

In [17]:
df_train = df.copy()
y_train = df_train.target.copy()
df_train = df_train.drop(columns=['target'])

### Feature selection

In [18]:
print('Feature Selection : Inicio')

Feature Selection : Inicio


In [19]:
cols = ['uc','index','date_fizcalizacion'] + cols_consumo
cols_for_feature_sel = [x for x in df_train.columns if x not in cols ]

In [20]:
len(cols_for_feature_sel)

170

In [21]:
# Si tenemos un dataset con muchos datos, podemos usar una muestra para hacer la seleccion de 
# variables

In [22]:
perc_sample = 1.0
df_train_fs = df_train.sample(frac=perc_sample, random_state=10) 
y_train_fs = y_train.loc[df_train_fs.index]

In [ ]:
# Nos quedamos con aquellas que no son constantes

In [23]:
%%time
select_by_constant = feature_selection_by_constant(df_train_fs, y_train_fs, cols_for_feature_sel, th=0.99)
pd.Series(sorted(select_by_constant)).to_csv('tmp/features_by_constante_12.csv', index=False)
print(f" # variables No constantes {len(select_by_constant)}")

 # variables No constantes 149
CPU times: user 430 ms, sys: 6.99 ms, total: 437 ms
Wall time: 435 ms


In [24]:
# De aquellas que no son constantes nos quedamos con aquellas que estan menos correlacionadas

In [25]:
%%time
select_by_corr = feature_selection_by_correlation(df_train_fs, y_train_fs, select_by_constant,method='pearson', th=0.90)
pd.Series(sorted(select_by_corr)).to_csv('tmp/features_by_corr_12.csv', index=False)
print(f" # variables No correlacionadas {len(select_by_corr)}")

Calculando Correlación Entre Variables
Eliminando Variables Altamente Correlacionadas
 # variables No correlacionadas 71
CPU times: user 2.75 s, sys: 62.9 ms, total: 2.82 s
Wall time: 2.81 s


In [71]:
# De aquellas que no son constantes nos quedamos con aquellas seleccionadas por MI

In [26]:
%%time
select_by_mi = feature_selection_by_mutual_information(df_train_fs[select_by_constant], y_train_fs,th=None)
pd.Series(sorted(select_by_mi)).to_csv('tmp/features_by_const_mi_12.csv', index=False)
print(f" # variables seleccionadas por MI {len(select_by_mi)}")

 # variables seleccionadas por MI 149
CPU times: user 23.3 s, sys: 113 ms, total: 23.4 s
Wall time: 23.4 s


In [75]:
# De aquellas que no son constantes nos quedamos con aquellas seleccionadas por Boruta version 1

In [83]:
%%time
select_by_boruta = feature_selection_by_boruta(df_train_fs[select_by_constant], y_train_fs, N=4)
pd.Series(sorted(select_by_boruta)).to_csv('tmp/features_by_const_boruta_12.csv', index=False)
print(f" # variables seleccionadas por Boruta v1 {len(select_by_boruta)}")

100%|██████████| 4/4 [04:25<00:00, 66.39s/it]

 # variables seleccionadas por Boruta v1 51
CPU times: user 1h 10min 12s, sys: 36.2 s, total: 1h 10min 49s
Wall time: 4min 25s


In [84]:
# De aquellas que no son constantes nos quedamos con aquellas seleccionadas por Boruta version 2

In [87]:
%%time
select_by_boruta2 = feature_selection_by_boruta2(df_train_fs[select_by_constant], y_train_fs)
pd.Series(sorted(select_by_boruta2)).to_csv('tmp/features_by_const_boruta2_12.csv', index=False)
print(f" # variables seleccionadas por Boruta v2 {len(select_by_boruta2)}")

  0%|          | 0/5 [00:00<?, ?it/s]

24745 6187


 20%|██        | 1/5 [00:59<03:56, 59.11s/it]

24745 6187


 40%|████      | 2/5 [01:58<02:57, 59.17s/it]

...

24746 6186


 80%|████████  | 4/5 [03:49<00:56, 56.22s/it]

24746 6186


100%|██████████| 5/5 [04:39<00:00, 55.89s/it]

 # variables seleccionadas por Boruta v2 5
CPU times: user 1h 1min 35s, sys: 33.9 s, total: 1h 2min 8s
Wall time: 4min 39s


In [88]:
print('Feature Selection : Fin')

Feature Selection : Fin


# Guardar variables tsfel en formato json

**Todas las variables que calcula TSFEL en formato json:**

 [Json - Features](https://github.com/fraunhoferportugal/tsfel/blob/master/tsfel/feature_extraction/features.json)   

In [99]:
tsfel_selected = list(set([x.split('_')[1] for x in select_by_boruta2 if x.split('_')[0]=='0' ]))
# Opening JSON file
with open("tsfel_total_features.json") as f:
    tsfel_features = json.load(f)
    
# seleccionamos
tsfel_new_features = dict()
for d in ['spectral','statistical','temporal']:
    dict_domain = dict(tsfel_features[d])
    for i in tsfel_features[d]:
        if i not in tsfel_selected:
            del dict_domain[i]
    
    tsfel_new_features[d] = dict_domain
    
# Data to be written
with open("tmp/tsfel_constantes_boruta2_12_features.json", "w") as outfile:
    json.dump(tsfel_new_features, outfile) 